In [1]:
# IMPORTAR BIBLIOTECAS E CARREGAR DADOS

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#para ignorar um futurewarning por questão estética
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Carregar os dados
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv").values.ravel()
y_test = pd.read_csv("y_test.csv").values.ravel()

print("Dados carregados com sucesso!")
print("Tamanho de treino:", X_train.shape, "| Tamanho de teste:", X_test.shape)


Dados carregados com sucesso!
Tamanho de treino: (3539, 13) | Tamanho de teste: (885, 13)


In [2]:
# CRIAR UMA FUNÇÃO PARA TREINAR E AVALIAR OS MODELOS

def avaliar_modelo(modelo, X_train, X_test, y_train, y_test):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"=== {modelo.__class__.__name__} ===")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

    return {
        'Modelo': modelo.__class__.__name__,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-score': f1
    }


DEFINIÇÃO DOS HIPER-PARAMETROS USADOS A BAIXO:
Os modelos foram instanciados com os hiperparâmetros mostrados no código-fonte do experimento, sem uso de busca automatizada de hiperparâmetros. As instâncias utilizadas foram: LogisticRegression(max_iter=1000), KNeighborsClassifier(n_neighbors=5), RandomForestClassifier(n_estimators=100, random_state=42) e GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42). Parâmetros não explicitados no código permaneceram com os valores padrão da biblioteca scikit-learn (por exemplo, solver='lbfgs' em LogisticRegression, criterion='gini' e max_depth=None em RandomForestClassifier, e profundidade de árvore max_depth=3 em GradientBoostingClassifier).

A escolha dessas configurações foi deliberada e guiada por princípios metodológicos: (i) simplicidade e reprodutibilidade — usar configurações padrão reduz o número de variáveis experimentais e facilita a reprodutibilidade; (ii) trade-off bias/variance — valores como n_estimators=100 para ensembles (Random Forest e Gradient Boosting) fornecem robustez e redução de variância sem custo computacional excessivo; (iii) robustez em dados tabulares — k=5 no KNN é um compromisso clássico entre sensibilidade a ruído e preservação de fronteiras de decisão; (iv) garantir convergência numérica — max_iter=1000 na regressão logística previne término prematuro do otimizador em presença de variáveis correlacionadas. Essas decisões foram validadas empiricamente por execuções exploratórias locais, priorizando interpretações estáveis das métricas (precision, recall, F1) em vez de otimização exaustiva.


Observação: Após a aplicação e avaliação inicial dos modelos, aquele que apresentar o melhor desempenho será selecionado para um processo de otimização utilizando o método GridSearchCV no notebook do dia 4. Essa estratégia busca equilibrar a qualidade do ajuste com a eficiência computacional, uma vez que o GridSearch demanda alto custo de processamento. Assim, a busca exaustiva por hiperparâmetros será restrita apenas ao modelo com maior potencial de desempenho.

In [3]:
# EFETIVAMENTE TREINAR E AVALIAR OS 4 MODELOS

resultados = []

# 1. Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
resultados.append(avaliar_modelo(log_reg, X_train, X_test, y_train, y_test))

# 2. K-Nearest Neighbors
knn = KNeighborsClassifier(n_neighbors=5)
resultados.append(avaliar_modelo(knn, X_train, X_test, y_train, y_test))

# 3. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
resultados.append(avaliar_modelo(rf, X_train, X_test, y_train, y_test))

# 4. Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
resultados.append(avaliar_modelo(gb, X_train, X_test, y_train, y_test))


=== LogisticRegression ===
Accuracy:  0.8734
Precision: 0.8726
Recall:    0.8734
F1-score:  0.8702

Matriz de Confusão:
[[568  33]
 [ 79 205]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       601
           1       0.86      0.72      0.79       284

    accuracy                           0.87       885
   macro avg       0.87      0.83      0.85       885
weighted avg       0.87      0.87      0.87       885

--------------------------------------------------
=== KNeighborsClassifier ===
Accuracy:  0.8486
Precision: 0.8478
Recall:    0.8486
F1-score:  0.8428

Matriz de Confusão:
[[566  35]
 [ 99 185]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       601
           1       0.84      0.65      0.73       284

    accuracy                           0.85       885
   macro avg       0.85      0.80      0.81       885
weig

In [4]:
# COMPARAR OS RESULTADOS DE FORMA ORGANIZADA EM UMA TABELA

resultados_df = pd.DataFrame(resultados)
resultados_df = resultados_df.sort_values(by="Recall", ascending=False)
display(resultados_df)


,Modelo,Accuracy,Precision,Recall,F1-score
3,GradientBoostingClassifier,0.881356,0.880066,0.881356,0.879282
0,LogisticRegression,0.873446,0.872586,0.873446,0.870203
2,RandomForestClassifier,0.863277,0.861496,0.863277,0.860213
1,KNeighborsClassifier,0.848588,0.847849,0.848588,0.842801


In [5]:
# ESCOLHA E JUSTIFICATIVA DO MODELO FINAL

# Identificar o melhor modelo com base no Recall e F1-score
melhor_modelo = resultados_df.sort_values(by=["Recall", "F1-score"], ascending=False).iloc[0]
modelo_escolhido = melhor_modelo["Modelo"]

print("=== Modelo selecionado para otimização ===")
print(f"Modelo escolhido: {modelo_escolhido}")
print(f"Recall: {melhor_modelo['Recall']:.3f}")
print(f"F1-score: {melhor_modelo['F1-score']:.3f}\n")

# Definir o modelo final para as próximas análises
from sklearn.ensemble import GradientBoostingClassifier

modelo_final = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

print("✅ Modelo Gradient Boosting definido como modelo final para otimização.")


=== Modelo selecionado para otimização ===
Modelo escolhido: GradientBoostingClassifier
Recall: 0.881
F1-score: 0.879

✅ Modelo Gradient Boosting definido como modelo final para otimização.


JUSTIFICATIVA ESCOLHA DE MODELO PARA OTIMIZAÇÃO NO DIA 4

O modelo escolhido foi o Gradient Boosting Classifier, pois apresentou o melhor equilíbrio entre Recall e F1-score.
No contexto do problema de evasão acadêmica, o Recall é uma métrica crucial, já que representa a capacidade do modelo
de identificar corretamente os alunos que estão em risco de evadir. 

Embora outros modelos tenham apresentado resultados próximos, o Gradient Boosting demonstrou desempenho consistente
e boa capacidade de generalização, sendo uma excelente escolha para a etapa de otimização de hiperparâmetros
que será realizada no Dia 4.


In [6]:
# NESSA E NA CELULA ABAIXO ANALISAMOS SE HOUVE OVERFITTING NO MODELO GRADIENT BOOSTING

# AVALIAÇÃO DE OVERFITTING: COMPARAÇÃO TREINO VS TESTE 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Treinar o modelo final no conjunto de treino
modelo_final.fit(X_train, y_train)

# Fazer previsões no treino e no teste
y_pred_train = modelo_final.predict(X_train)
y_pred_test = modelo_final.predict(X_test)

# Função para calcular métricas
def calc_metricas(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='weighted'),
        'Recall': recall_score(y_true, y_pred, average='weighted'),
        'F1-score': f1_score(y_true, y_pred, average='weighted')
    }

# Calcular métricas para treino e teste
metricas_train = calc_metricas(y_train, y_pred_train)
metricas_test = calc_metricas(y_test, y_pred_test)

# Exibir comparação
print("=== Comparação treino vs teste ===")
print(f"Treino → Accuracy: {metricas_train['Accuracy']:.3f}, Recall: {metricas_train['Recall']:.3f}, F1: {metricas_train['F1-score']:.3f}")
print(f"Teste  → Accuracy: {metricas_test['Accuracy']:.3f}, Recall: {metricas_test['Recall']:.3f}, F1: {metricas_test['F1-score']:.3f}")

# Análise automática
dif_recall = metricas_train['Recall'] - metricas_test['Recall']
dif_f1 = metricas_train['F1-score'] - metricas_test['F1-score']

if dif_recall > 0.05 or dif_f1 > 0.05:
    print("\n⚠️ Há indícios de overfitting: o modelo performa significativamente melhor no treino do que no teste.")
else:
    print("\n✅ O modelo generaliza bem — não há sinais fortes de overfitting.")


=== Comparação treino vs teste ===
Treino → Accuracy: 0.877, Recall: 0.877, F1: 0.875
Teste  → Accuracy: 0.881, Recall: 0.881, F1: 0.879

✅ O modelo generaliza bem — não há sinais fortes de overfitting.


In [7]:
# NESSA CELULA ANALISAMOS SE HOUVE OVERFITTING NO MODELO GRADIENT BOOSTING

# AVALIAÇÃO DE OVERFITTING: VALIDAÇÃO CRUZADA PARA CONFIRMAR ESTABILIDADE DO MODELO
from sklearn.model_selection import cross_val_score
import numpy as np

# Executar validação cruzada no conjunto de treino
scores_recall = cross_val_score(modelo_final, X_train, y_train, cv=5, scoring='recall_weighted')
scores_f1 = cross_val_score(modelo_final, X_train, y_train, cv=5, scoring='f1_weighted')

# Exibir resultados
print("=== Validação cruzada (5 folds) ===")
print(f"Recall por fold: {np.round(scores_recall, 3)}")
print(f"Recall médio: {scores_recall.mean():.3f} ± {scores_recall.std():.3f}")
print(f"F1 por fold: {np.round(scores_f1, 3)}")
print(f"F1 médio: {scores_f1.mean():.3f} ± {scores_f1.std():.3f}")

# Análise automática
if scores_f1.std() < 0.02:
    print("\n✅ O desempenho é estável entre os folds — o modelo é consistente.")
elif scores_f1.std() < 0.05:
    print("\n⚠️ Pequena variação entre folds — desempenho razoavelmente estável.")
else:
    print("\n❗ Alta variação entre folds — possível instabilidade ou overfitting.")


=== Validação cruzada (5 folds) ===
Recall por fold: [0.856 0.857 0.87  0.843 0.87 ]
Recall médio: 0.859 ± 0.010
F1 por fold: [0.853 0.855 0.867 0.841 0.866]
F1 médio: 0.857 ± 0.010

✅ O desempenho é estável entre os folds — o modelo é consistente.


AMBAS AS AVALIAÇÃOES DE OVERFITTING FORAM SATISFATÓRIAS 
................................................................................................................................................................................................................................................................

In [8]:
# SALVANDO A AVALIAÇÃO DO MODELO ESCOLHIDO EM UM JSON

import json

avaliacao = {
    "Treino_vs_Teste": {"Train": metricas_train, "Test": metricas_test},
    "CrossValidation": {
        "Recall_medio": scores_recall.mean(),
        "Recall_std": scores_recall.std(),
        "F1_medio": scores_f1.mean(),
        "F1_std": scores_f1.std()
    }
}

with open("avaliacao_gradient_boosting.json", "w") as f:
    json.dump(avaliacao, f, indent=4)

print("✅ Avaliação salva em 'avaliacao_gradient_boosting.json'")


✅ Avaliação salva em 'avaliacao_gradient_boosting.json'


In [9]:
# SALVANDO OS RESULTADOS EM UM ARQUIVO .csv

resultados_df.to_csv("resultados_modelagem.csv", index=False)
print("Resultados salvos em 'resultados_modelagem.csv'.")


Resultados salvos em 'resultados_modelagem.csv'.
